# Use NumPy to recognize handwritten digits

In [1]:
from PIL import Image
import numpy as np

Download the data. In this case, MNIST dataset.

In [2]:
import os
import urllib
from urllib.request import urlretrieve
dataset = 'mnist.pkl.gz'
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(a*b*100.0/c), end="")
    
if not os.path.isfile(dataset):
        origin = "https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz"
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset, reporthook=reporthook)

In [3]:
import gzip
import pickle
with gzip.open(dataset, 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

## Q
First, check shape and dtype of the dataset.

In [4]:
train_set

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([5, 0, 4, ..., 8, 4, 8], dtype=int64))

In [5]:
%run -i q_see_mnist_data.py

train_set[0]:      shape=(50000, 784)	 dtype=float32
train_set[1]:      shape=(50000,)	 dtype=int64
validation_set[0]: shape=(10000, 784)	 dtype=float32
validation_set[1]: shape=(10000,)	 dtype=int64
test_set[0]:       shape=(10000, 784)	 dtype=float32
test_set[1]:       shape=(10000,)	 dtype=int64


# Supervised Learning

<img src="supervised.svg" />
vs
<img src="supervised2.svg" />
See also Chinese room

In [6]:
train_X, train_y = train_set
test_X, test_y = test_set

### Look into the content. What is in `y`?

In [7]:
train_y[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=int64)

In [8]:
train_X.shape

(50000, 784)

In [9]:
train_X[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
# Let's check the first 20 items in the training data
train_y[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=int64)

### What is in `X`?

In [11]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# show the first 20 samples in training X
showX(train_X[120:150])

In [12]:
showX(test_X[:10])

In [13]:
test_y.shape

(10000,)

# nearest neighbor

### Let's start from the simplest mehods

Always start from the obvious, simple, boring methods, 

For instance, in order to decide the digit in a handwritten sample, can we simply identify the most similar image in the training data, and assume they represents the same digit?

It may appear naive, but why not give it a shot?

But what exactly does similarity mean? You know, it is kind of tricky to define the similarity of two images. Some approaches, such as SSIM are related to human visual neurobiology and perception, and some methods like LPIPS employ neural networks. 

Again, always begin with the obvious, simple, boring methods. Let us using sqauare error to measure the differences between two images.

## Q 
How to calculate the square difference between two vectors u and v?
```python
u = train_X[0]
v = train_X[1]
```

In [14]:
showX(train_X[:2])

In [15]:
# %load q_square_error.py
u = train_X[0]
v = train_X[1]
print ( ((u - v)**2).sum() )
print ( np.linalg.norm(u-v)**2 )

86.94919
86.949183020698


In [16]:
showX(train_X[0:2])

In [17]:
train_X.min()

0.0

In [18]:
np.abs(test_X[0] - train_X[1]).sum()

134.97266

In [19]:
showX(test_X[1:2])

In [20]:
target_id = 2217
X = test_X[target_id]
print("target:", test_y[target_id])
showX(X)
best_dst, best_i = 100000, -1
for i in range(50000):
    dst = ((X - train_X[i])**2).sum()
    if dst < best_dst:
        best_dst, best_i = dst, i
        print(i, dst, train_y[i])
        showX(np.array([X, train_X[i]] ))

target: 0


0 123.71152 5


1 109.54404 0


18 108.45303 6


48 102.109726 9


...

429 47.079758 0


1028 45.554276 0


1078 40.134384 0


1530 33.41208 0


1590 22.06726 0


In [21]:
showX(train_X[1609])

## Q 
* Show test_X[0]
* Find a sample in  `train_X` which is most similar to `test_X[0]`.
* display the image of that sample
* check the corresponding  `train_y`
* see if it is the same as test_y[0]

In [22]:
# %load q_find_nn_0.py
# 顯示 test_X[0]
showX(test_X[0])
# 計算方差
_ = ((train_X - test_X[0])**2).sum(axis=1)
# 找出方差最小的 index
idx = _.argmin()
print("train_X[{}]".format(idx))
showX(train_X[idx])
print("train_X[{}] = {}".format(idx, train_y[idx]))
print("test_y[0] =", test_y[0])


train_X[38620]


train_X[38620] = 7
test_y[0] = 7


## Q

Take the first 10, 100 samples from `test_X` and do the same. What is the accuracy of this method?

In [23]:
# %load q_find_nn_10.py
from tqdm.auto import tqdm
N = 100
correct = 0
for i in tqdm(range(N)):
    # 顯示 test_X[i]
    #if i%30==0:
    #    print("test_X[{}]".format(i))
    #showX(test_X[i])
    # 計算方差
    _ = ((train_X - test_X[i])**2).sum(axis=1)
    # 找出方差最小的 index
    idx = _.argmin()
    #print("train_X[{}]".format(idx))
    #showX(train_X[idx])
    #print("train_X[{}] = {}".format(idx, train_y[idx]))
    #print("train_X[{}] = {}".format(i, test_y[i]))
    if train_y[idx] == test_y[i]:
        correct+=1
print("Accuracy", correct/N)


C:\Users\tzerj\anaconda3\envs\test1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████| 100/100 [00:10<00:00,  9.52it/s]

Accuracy 1.0


## Q
What if train_X cotains only 500 samples?

use reshaping,  broadcasting to caculate the accuracy for `test_X[:100]`.

Hint: Can use `np.expand_dims` instead of `np.reshape`.

In [24]:
# !may use too much memory
#%run -i q_small_data.py
# accuracy: 85%

In [25]:
#(u-v) ** 2 sum
X = test_X[0]
((train_X - X) ** 2).sum(axis=1)

array([ 87.58296 , 107.205826,  88.42671 , ...,  99.16844 ,  93.548874,
       104.29837 ], dtype=float32)

In [26]:
U = train_X.reshape((50000, 1, 784))
U.shape

(50000, 1, 784)

In [27]:
V = test_X[:30].reshape((1, 30, 784))
V.shape

(1, 30, 784)

In [28]:
R = ((U-V)**2) .sum(axis=2)
R.shape

(50000, 30)

In [29]:
K = 1024
M = 1024*K
G = 1024*M
(50000 * 100 * 784* 4)/G

14.603137969970703

In [30]:
(U-V).shape

(50000, 30, 784)

In [31]:
(train_y[R.argmin(axis=0)] == test_y[:10]).mean()

ValueError: operands could not be broadcast together with shapes (30,) (10,) 

In [ ]:
test_y[:10]

## Q
What about using other distance functions? e.g.  `np.abs(...).sum()`

### Use inner product instead of square difference
$$
\begin{align*}
\left\Vert \mathbf{u}-\mathbf{v}\right\Vert ^{2} & =\left(\mathbf{u}-\mathbf{v}\right)\cdot\left(\mathbf{u}-\mathbf{v}\right)\\
 & =\left\Vert \mathbf{u}\right\Vert ^{2}-2\mathbf{u}\cdot\mathbf{v}+\left\Vert \mathbf{v}\right\Vert ^{2}\\
\end{align*}
$$


In [32]:
# workaround
train_X0 = train_X
test_X0 = test_X

In [33]:
# normalize the data
train_X  = train_X0 / np.linalg.norm(train_X, axis=1, keepdims=True)
test_X  = test_X0 / np.linalg.norm(test_X, axis=1, keepdims=True)

In [37]:
# matrix multiplication == pairwise computation of inner product
A = test_X @ train_X.T
print(A.shape)

(10000, 50000)


In [38]:
A.argmax(axis=1)

array([44566, 28882, 15224, ...,  3261,  1311, 22424], dtype=int64)

In [39]:
predict_y = train_y[A.argmax(axis=1)]
predict_y

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [40]:
# Show first 20 samples of  Test X
showX(test_X0[:20])

In [41]:
# predicted Y
predict_y[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=int64)

In [42]:
# First 20 samples of Test y
test_y[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=int64)

In [43]:
# accuracy
(predict_y == test_y).mean()

0.9708

### Employ PCA to reduce dimensions
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


In [46]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
train_X = pca.fit_transform(train_set[0])
test_X = pca.transform(test_set[0])

In [47]:
train_X.shape

(50000, 100)

In [31]:
np.mean((train_set[0][0]-train_set[0][1])**2)

0.110904574

In [32]:
np.mean((train_X[0]-train_X[1])**2)

0.11090599

In [33]:
m = train_set[0].mean(axis=0)
m.shape

(784,)

In [34]:
train_X[0]@train_X[1]

10.671793

In [35]:
(train_set[0][0]-m)@(train_set[0][1]-m)

10.671726

In [36]:
train_X /= np.linalg.norm(train_X, axis=1, keepdims=True)
test_X /= np.linalg.norm(test_X, axis=1, keepdims=True)

In [37]:
# using matrix multiplication
A = test_X @ train_X.T

In [38]:
predict_y = train_y[A.argmax(axis=1)]

In [39]:
# accuracy
(predict_y == test_y).mean()

0.9669

## Q
* Try other settings of PCA
* Check the differences
* use sklearn knn